In [1]:

import datetime as dt
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import h2o
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Polygon
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from statsmodels.tools.tools import add_constant

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from h2o.automl import H2OAutoML
from tpot import TPOTClassifier
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC


In [2]:
accidents_sub = pd.read_csv("C:\\Users\\reza3\\OneDrive\\Desktop\\AIT\\Machine learning\\group project\\data\\sub_accidents.csv")

In [ ]:
accidents_sub.shape

In [3]:
accidents_sub.drop(columns="Unnamed: 0", inplace=True)

In [ ]:
accidents_sub.columns

In [5]:
X = accidents_sub.drop(columns=['ID', 'Severity', 'Delay(min)', 'Street', 'City', 'County', 'State', 'Zipcode',
       'Start_Year', 'Start_Month', 'Start_Day',
       'Start_time','Distance(km)', 'Delay_ln', 'Severity_new'])
Y = accidents_sub.loc[:, ['Severity', 'Severity_new', 'Delay(min)', 'Delay_ln', 'Distance(km)']]


In [6]:
# fxing imbalance by under sampling (random under sampling)
rus = RandomUnderSampler(random_state=42, replacement=False)

# fit predictor and target varialbe
X_rus, Y_rus = rus.fit_resample(X, Y["Severity_new"])

X.shape, Y.shape, X_rus.shape, Y_rus.shape

((4016917, 22), (4016917, 5), (912780, 22), (912780,))

In [ ]:
# fxing imbalance by under sampling (Tomek links)
tl = TomekLinks(sampling_strategy='majority', n_jobs=-1)

# fit predictor and target variable
X_tl, Y_tl = tl.fit_resample(X, Y["Severity_new"])

In [7]:
Y_rus.value_counts()

Severity_new
0    456390
1    456390
Name: count, dtype: int64

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X_rus, Y_rus, test_size = 0.10, random_state=370)

assert len(X_train)  == len(Y_train)
assert len(X_test)   == len(Y_test)
X.shape, Y.shape,X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((4016917, 22), (4016917, 5), (821502, 22), (91278, 22), (821502,), (91278,))

I tried to fit a regression model with Delay and distance as labels but no appropriate model have found.

In [ ]:
'''model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=370)

model.fit(X_train, Y_train["Distance(km)"])

# Make predictions on the validation set
y_train_pred = model.predict(X_train)

# Calculate metrics
mse = mean_squared_error(Y_train["Distance(km)"], y_train_pred)
r2 = r2_score(Y_train["Distance(km)"], y_train_pred)
print(f'Mean Squared Error on Test Set: {mse}')
print(f'R_squared on Test Set: {r2}')'''




In [ ]:
lr = LogisticRegression(random_state=370, n_jobs=-1)
rf = RandomForestClassifier(random_state=370, n_jobs=-1)
sgb = xgboost.XGBClassifier(max_depth=1, n_estimator=100, n_jobs=-1)
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate=0.001, random_state=370)

models = [ lr, rf, sgb, ada]

#perform cross validation using KFold
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits = 5, shuffle = True, random_state=370)

for model in models:
    score = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy', n_jobs=-1)  #f1, recall, precision, accuracy
    np.set_printoptions(precision = 2)
    print("Model: ", model, "Scores: ", score, "- Scores mean: ", score.mean(), "- Scores std (lower better):", score.std()) 
    fit = model.fit(X_train, Y_train)
    yhat = fit.predict(X_test)
    print(confusion_matrix(Y_test, yhat))
    print('ROCAUC score:',roc_auc_score(Y_test, yhat))      
    print(classification_report(Y_test, yhat))

In [ ]:
ada_model = ada.fit(X_train, Y_train["Severity_new"])
yhat = ada_model.predict(X_test)
print(classification_report(Y_test["Severity_new"], yhat))

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {"n_estimators": [10], 
              "criterion": ["gini", "entropy"],
              "max_depth": 10}
model = RandomForestClassifier()
model.fit(X_train, Y_train["Severity"])
yhat = model.predict(X_test)

print(classification_report(Y_test["Severity"], yhat))'''

'''grid = GridSearchCV(model, param_grid, refit=True)
grid.fit(X_train, Y_train["Severity"])

print(grid.best_params_)

yhat = grid.predict(X_test["Severity"])

print(classification_report(Y_test["Severity"], yhat))'''
'''from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import  classification_report

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, Y_train["Severity"])
y_pred = ada_clf.predict(X_test)
print("Ada score: ", accuracy_score(Y_test["Severity"], y_pred))
print(classification_report(Y_test["Severity"], y_pred))'''
'''



#not improved after 2 iterations
xgb_reg.fit(X_train, Y_train["Distance(km)"],
                eval_set=[(X_test, Y_test["Distance(km)"])])
y_pred = xgb_reg.predict(X_test)
mse = mean_squared_error(Y_test["Distance(km)"], y_pred)
r2 = r2_score(Y_test["Distance(km)"], y_pred)

print("MSE:", mse)  #notice we are using mse while xgb uses root mse
print("R2:", r2)'''
'''

import torch

from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

inputs = torch.from_numpy(X_train.values)
target = torch.from_numpy(Y_train["Delay(min)"].values)
print(inputs.size())
print(target.size())
from torch.utils.data import TensorDataset
# Define dataset
train_ds = TensorDataset(inputs, target)
from torch.utils.data import DataLoader
# Define data loader
batch_size = 500
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
# Build the ANN model

model = Sequential()

# Input layer
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='LeakyReLU'))

# Hidden layers
model.add(Dense(64, activation='LeakyReLU'))
model.add(Dense(48, activation='LeakyReLU'))
model.add(Dense(24, activation='LeakyReLU'))
model.add(Dense(12, activation='LeakyReLU'))

# Output layer
model.add(Dense(1, activation='linear'))  # Linear activation for regression

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, Y_train["Distance(km)"], epochs=1, batch_size=100, validation_data=(X_test, Y_test["Distance(km)"]))
yhat = model.predict(X_test)
r2 = r2_score(Y_test["Distance(km)"], yhat)
weights_and_biases = model.get_weights()
# Evaluate the model
loss = model.evaluate(X_test, Y_test["Distance(km)"])
print(f'Mean Squared Error on Test Set: {loss:.2f}')
print(r2)
'''
from sklearn.cluster import KMeans

#sum of squared distances
ssd = []
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k, n_init='auto')
    kmeans.fit(X)
    ssd.append(kmeans.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), ssd)
plt.xticks(range(2, 20))
plt.grid(True)
plt.ylabel("within cluster variation")
plt.xlabel("k")
plt.title('Elbow curve')

plt.annotate('elbow', xy=(4.3, 220), xytext=(5, 600),  #xytext ---> xy
            arrowprops=dict(arrowstyle="->"))